In [1]:
from ree_descarga import descarga_ree
from ree_compactador import compactador_ree
import pandas as pd
from datetime import timedelta
from pathlib import Path
from collections import defaultdict
import shutil
import os


In [2]:


def comprobar_fechas_faltantes_por_tabla(path_base):
    tablas = ["almacenamiento", "demanda", "emision", "generacion"]
    fechas_por_tabla = {}

    for tabla in tablas:
        path_tabla = Path(path_base) / tabla
        fechas = set()

        # Buscar todos los archivos Parquet en todas las particiones year=XXXX/
        for archivo in path_tabla.rglob("*.parquet"):
            try:
                df = pd.read_parquet(archivo, columns=["Hora"])
                df["fecha"] = df["Hora"].dt.date
                fechas.update(df["fecha"].unique())
            except Exception as e:
                print(f"Error leyendo {archivo}: {e}")

        fechas_por_tabla[tabla] = fechas

    # Todas las fechas que aparecen al menos en una tabla
    fechas_totales = set.union(*fechas_por_tabla.values())

    # Buscar en qué tablas falta cada fecha
    fechas_faltantes = defaultdict(list)
    for fecha in fechas_totales:
        for tabla in tablas:
            if fecha not in fechas_por_tabla[tabla]:
                fechas_faltantes[fecha].append(tabla)

    # Mostrar resultado
    if not fechas_faltantes:
        print("No faltan fechas en ninguna tabla.")
    else:
        print("Faltan fechas en las siguientes tablas:")
        for fecha, tablas_que_faltan in sorted(fechas_faltantes.items()):
            print(f"{fecha} falta en: {', '.join(tablas_que_faltan)}")

# Uso:
comprobar_fechas_faltantes_por_tabla("../data_parquet")

No faltan fechas en ninguna tabla.


In [3]:
list_tabla = ['demanda', 'emision', 'generacion', 'almacenamiento']
list_df = []
for tabla in list_tabla:
    df = pd.read_parquet(f"../data_parquet/{tabla}", columns=['Hora'])
    duplicados = df[df["Hora"].duplicated()]
    print(duplicados)
    list_df.append([tabla,df])

print(len(list_df))


Empty DataFrame
Columns: [Hora]
Index: []
Empty DataFrame
Columns: [Hora]
Index: []
Empty DataFrame
Columns: [Hora]
Index: []
Empty DataFrame
Columns: [Hora]
Index: []
4


In [4]:


list_tabla = ['demanda', 'emision', 'generacion', 'almacenamiento']
horas_por_tabla = {}

def comprobar_duplicados_faltantes(list_tabla, path="../data_parquet/"):
    for tabla in list_tabla:
        df = pd.read_parquet(f"{path}{tabla}", columns=['Hora'])

        # Verificar duplicados (opcional)
        duplicados = df[df["Hora"].duplicated()]
        if not duplicados.empty:
            print(f"Duplicados en {tabla}:\n", duplicados)

        # Guardar las horas únicas como set
        horas_unicas = set(df["Hora"])
        horas_por_tabla[tabla] = horas_unicas

    # Unir todas las horas únicas que aparecen en alguna tabla
    todas_las_horas = set.union(*horas_por_tabla.values())

    # Para cada hora, comprobar en qué tablas falta
    faltantes_por_hora = {}

    for hora in todas_las_horas:
        faltan = [tabla for tabla in list_tabla if hora not in horas_por_tabla[tabla]]
        if faltan:
            faltantes_por_hora[hora] = faltan

    # Mostrar resultados
    for hora, tablas_faltantes in sorted(faltantes_por_hora.items()):
        print(f"{hora} falta en: {', '.join(tablas_faltantes)}")

    print(f"\nTotal de horas con datos incompletos: {len(faltantes_por_hora)}")

    return faltantes_por_hora

faltantes_por_hora = comprobar_duplicados_faltantes(list_tabla)

2021-05-12 12:30:00 falta en: emision, generacion, almacenamiento
2021-05-12 12:35:00 falta en: emision, generacion, almacenamiento
2021-05-12 12:40:00 falta en: emision, generacion, almacenamiento
2021-05-12 12:45:00 falta en: emision, generacion, almacenamiento
2021-05-12 12:50:00 falta en: emision, generacion, almacenamiento
2021-05-12 12:55:00 falta en: emision, generacion, almacenamiento
2021-05-12 13:00:00 falta en: emision, generacion, almacenamiento
2021-05-12 13:05:00 falta en: emision, generacion, almacenamiento
2021-05-12 13:10:00 falta en: emision, generacion, almacenamiento
2021-05-12 13:15:00 falta en: emision, generacion, almacenamiento
2021-05-12 13:20:00 falta en: emision, generacion, almacenamiento
2021-05-12 13:25:00 falta en: emision, generacion, almacenamiento
2021-05-12 13:30:00 falta en: emision, generacion, almacenamiento
2021-05-12 13:35:00 falta en: emision, generacion, almacenamiento
2021-05-12 13:40:00 falta en: emision, generacion, almacenamiento
2021-05-12

In [5]:
list_days = []
for hora, tablas_faltantes in sorted(faltantes_por_hora.items()):
    list_days.append(hora.date())

# Contar ocurrencias por día
from collections import Counter
contador_dias = Counter(list_days)

# Mostrar resultados
for dia, cantidad in contador_dias.items():
    print(f"El día {dia} tiene {cantidad} faltantes.")

El día 2021-05-12 tiene 138 faltantes.
El día 2021-05-17 tiene 180 faltantes.
El día 2021-09-05 tiene 9 faltantes.
El día 2022-01-24 tiene 156 faltantes.
El día 2022-01-25 tiene 131 faltantes.
El día 2022-10-25 tiene 1 faltantes.
El día 2023-04-14 tiene 3 faltantes.
El día 2023-05-17 tiene 7 faltantes.
El día 2023-10-11 tiene 2 faltantes.
El día 2025-06-05 tiene 10 faltantes.
El día 2025-06-17 tiene 1 faltantes.


In [6]:
# completar las tablas parquet de los dias y registros de 5 minutos faltantes, rellenando los valores con la media del valor anteriores y posterior


for tabla in list_tabla:
    df = pd.read_parquet(f"../data_parquet/{tabla}")
    
    # Asegurarse de que 'year' sea de tipo numérico
    if 'year' in df.columns and df['year'].dtype == 'category':
        df['year'] = df['year'].astype('int32')
    
    df.set_index('Hora', inplace=True)
    df = df.asfreq('5min')  # Asegurarse de que todos los registros de 5 minutos estén presentes
    df.sort_index(inplace=True)
    
    # Interpolación para columnas numéricas
    numeric_columns = df.select_dtypes(include=['float64', 'int32', 'int64']).columns
    df[numeric_columns] = df[numeric_columns].interpolate(method='time')
    
    df.reset_index(inplace=True)

    # Borramos el directorio antes de escribir si existe
    if os.path.exists(f"../data_parquet_clean/{tabla}"):
        shutil.rmtree(f"../data_parquet_clean/{tabla}")

    # Guardar el DataFrame corregido
    df.to_parquet(f"../data_parquet_clean/{tabla}", partition_cols=['year'], index=False) 
    print(f"Tabla {tabla} corregida y guardada.")



# Verificar nuevamente las tablas para asegurarse de que no haya faltantes
faltantes_por_hora = comprobar_duplicados_faltantes(list_tabla,path="../data_parquet_clean/")
faltantes_por_hora



Tabla demanda corregida y guardada.
Tabla emision corregida y guardada.
Tabla generacion corregida y guardada.
Tabla almacenamiento corregida y guardada.

Total de horas con datos incompletos: 0


{}